# START THE EXPERIMENT !!

In [1]:
import numpy as np
import pandas as pd
import seaborn as sea
import matplotlib.pyplot as plt

plt.style.use('ggplot')
sea.set_style('whitegrid')
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# https://stackoverflow.com/questions/21971449/how-do-i-increase-the-cell-width-of-the-jupyter-ipython-notebook-in-my-browser

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

import os
import time
import copy
import random
import shutil
import meth_model_utils as u
import meth_model_classes as c

In [4]:
import warnings
warnings.simplefilter('ignore')

In [5]:
import os
import time
import copy
import shutil

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import utils
from torch.utils.data import Dataset, DataLoader

In [6]:
torch.use_deterministic_algorithms(True)
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":16:8"

# Setup device and seed

In [7]:
device = torch.device('cuda' if  torch.cuda.is_available() else 'cpu')

In [8]:
device

device(type='cuda')

In [9]:
seed = u.get_seed()
print('Seed = ', seed)
u.set_all_seeds(seed)

Seed =  324


---
---
---

# Setup dataframe

In [10]:
PATH = "D:/CANCER BIOLOGY/DATASET/TCGA/FROM Xena/"

In [11]:
df_luad = pd.read_csv(PATH+"meth_luad.csv", index_col=0)
df_lusu = pd.read_csv(PATH+"meth_lusu.csv", index_col=0)
df_lusu.drop(index=['MBD3L2'], axis=0, inplace=True)
df_final = u.meth_data_preprocess(df_luad, df_lusu)

In [12]:
df_final = df_final.sample(frac=1, random_state=seed).reset_index(drop=True)
labels = list(df_final['label'])
df_final.drop(columns=['label'], axis=1, inplace=True) ## drop column sample_id and label
columns = list(df_final.columns)

In [13]:
xtrain = df_final.to_numpy()
ytrain = labels

In [14]:
xtrain.shape

(828, 20115)

In [15]:
xtrain

array([[-0.2409, -0.1083,  0.2395, ...,  0.0487,  0.0487, -0.3371],
       [-0.3149, -0.0382,  0.1613, ..., -0.133 , -0.133 , -0.3876],
       [ 0.0038, -0.0716,  0.1159, ..., -0.0513, -0.0513, -0.3075],
       ...,
       [-0.4203, -0.4794,  0.2244, ...,  0.0016,  0.0016, -0.4379],
       [-0.4551, -0.4582,  0.1067, ..., -0.1569, -0.1569, -0.2953],
       [-0.4541, -0.4749, -0.0786, ...,  0.0435,  0.0435, -0.3795]])

In [16]:
df_final

,RBL2,VDAC3,ACTN1,ATP2A1,SFRP1,NIPA2,MAN1B1,TSEN34,LRRC16A,CNBP,...,BAGE5,BAGE,BAGE2,BAGE4,BAGE3,MIR637,LOC100130932,GOLGA8F,GOLGA8G,MIR7-1
0,-0.2409,-0.1083,0.2395,0.3893,-0.0479,0.3245,-0.4213,-0.3727,-0.3146,-0.4176,...,0.1955,0.1955,0.1955,0.1955,0.1955,0.3568,0.4044,0.0487,0.0487,-0.3371
1,-0.3149,-0.0382,0.1613,0.3379,0.0013,0.3428,-0.4294,-0.3522,-0.3440,-0.4229,...,0.0527,0.0527,0.0527,0.0527,0.0527,0.3562,0.4002,-0.1330,-0.1330,-0.3876
2,0.0038,-0.0716,0.1159,0.1238,-0.1988,0.3203,-0.4264,-0.3297,-0.3182,-0.3750,...,0.0774,0.0774,0.0774,0.0774,0.0774,0.3618,0.3936,-0.0513,-0.0513,-0.3075
3,-0.1457,-0.1677,0.1674,0.3513,-0.4500,0.3623,-0.4344,-0.3153,-0.3492,-0.3263,...,0.0343,0.0343,0.0343,0.0343,0.0343,0.3538,0.3837,-0.0495,-0.0495,-0.4226
4,-0.0836,-0.0579,0.0985,0.3155,-0.1373,0.3005,-0.4475,-0.2967,-0.3646,-0.4396,...,0.1695,0.1695,0.1695,0.1695,0.1695,0.3471,0.4030,-0.0526,-0.0526,-0.4085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823,-0.1859,0.0367,0.0753,0.3212,-0.1880,0.3145,-0.4332,-0.3189,-0.3267,-0.4030,...,0.1584,0.1584,0.1584,0.1584,0.1584,0.3179,0.3801,-0.0272,-0.0272,-0.3877
824,-0.4289,-0.4730,0.1825,-0.4401,-0.3812,-0.4896,-0.4896,-0.4319,-0.4663,-0.1633,...,0.1005,0.1005,0.1005,0.1005,0.1005,0.3420,0.3871,-0.2103,-0.2103,-0.1958
825,-0.4203,-0.4794,0.2244,-0.2298,-0.1448,-0.4858,-0.4867,-0.4549,-0.4691,0.1942,...,0.2590,0.2590,0.2590,0.2590,0.2590,0.4169,0.4268,0.0016,0.0016,-0.4379
826,-0.4551,-0.4582,0.1067,-0.3967,-0.3489,-0.4888,-0.4867,-0.4645,-0.4709,0.2420,...,-0.0901,-0.0901,-0.0901,-0.0901,-0.0901,0.3944,0.3996,-0.1569,-0.1569,-0.2953


---
---
---
---

# Start training autoencoder

In [17]:
input_dim = xtrain.shape[1]
epochs = 20
batch_size = 32
learning_rate = 0.000005
output_dim = 2048

In [18]:
train_dataset = c.LUNG_Meth(ytrain, xtrain)

In [19]:
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)

# Train and plot

In [20]:
u.set_all_seeds(seed)

In [21]:
model = c.METH_AutoEncoder(input_dim, output_dim)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [22]:
log_dict = u.train_encoder(
    num_epochs = epochs,
    model = model,
    optimizer = optimizer,
    device = device,
    train_loader = train_loader,
    skip_epoch_stats = False,
    logging_interval = 500,
    patience=5
)

  0%|          | 0/26 [00:00<?, ?it/s]

Epoch: 001/020 | Loss: 0.1312


  0%|          | 0/26 [00:00<?, ?it/s]

Epoch: 002/020 | Loss: 0.0555


  0%|          | 0/26 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
u.plot_training_loss(log_dict['train_loss_per_batch'], epochs)
plt.title("Autoencoder: Min loss={:.4f}".format(log_dict['final_loss']))
plt.savefig(PATH+"project_summary_seed_wise_meth/seed="+str(seed)+"/autoencoder", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
torch.save(model.state_dict(), PATH+"models/LUNG_METH_Autoencoder.kd")

## Compute pearson correlation b/w True_Dataset and Generated_Dataset

In [ ]:
u.set_all_seeds(seed)

In [ ]:
pearson_loader = DataLoader(train_dataset, len(train_dataset), shuffle=True)
pearson_correlation_coeff = u.pearson_correlation(model, pearson_loader, device)
print("Pearson Correlation Coefficient = ", pearson_correlation_coeff)

with open(r'D:/CANCER BIOLOGY/DATASET/TCGA/FROM Xena/project_summary_seed_wise_meth/seed=%s/pearson.corr'%str(seed), 'w') as fp:
    fp.write("%s\n" % pearson_correlation_coeff)

In [ ]:
from scipy.io.wavfile import read

fs, data = read('alert.wav', mmap=True)  # fs - sampling frequency
data = data.reshape(-1, 1)
import sounddevice as sd
sd.play(data, 44100)

---
---
---

# LEVEL 1  complete !